## Start

In [1]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)



___
### Tags: #Source #Partner

### Links: 
___

In [ ]:
# creating a table from s3

query_text = """--sql
    CREATE TABLE db1.uk_dir_partner
    (
        `id` Int32,
        `name`  String,
        `partner_uuid_uk` String,
        `created_at` String
    )
    ENGINE = S3('https://storage.yandexcloud.net/dwh-asgard/uk_dir_partner/uk_dir_partner.csv','CSVWithNames')
    """

ch.query_run(query_text)

In [ ]:
# creating a table for materialized view

query_text = """--sql
    CREATE TABLE db1.uk_dir_partner_ch
    (
        `id` Int32,
        `name`  String,
        `partner_uuid_uk` String,
        `created_at` String
    )
    ENGINE = MergeTree()
    ORDER BY partner_uuid_uk
    """

ch.query_run(query_text)

In [ ]:
# creating a materialized view

query_text = """--sql
    CREATE MATERIALIZED VIEW db1.uk_dir_partner_mv
    REFRESH EVERY 1 DAY OFFSET 3 HOUR RANDOMIZE FOR 1 HOUR TO db1.uk_dir_partner_ch AS
    SELECT
        *
    FROM db1.uk_dir_partner
    """

ch.query_run(query_text)

___
## Tools
___
### query

In [2]:
query_text = """--sql
    SELECT
        *
    FROM db1.uk_dir_partner_ch
    limit 10
    """

ch.query_run(query_text)

id,name,partner_uuid_uk,created_at
i64,str,str,str
68,"""Жилкомфорт 105 кв""","""0076e1b0-72c3-4783-9306-93d299…","""2024-03-06 02:42:16"""
345,"""УК ""НЭК ЭНЕРГО""""","""008d558d-fa9b-499f-b0a4-b4a00b…","""2025-06-27 13:19:04"""
284,"""Физкультурная 9/3""","""00b3faa7-ed4f-4ee5-93a1-7f0461…","""2025-03-18 00:30:27"""
37,"""УК""","""0129eea8-c15b-4086-92c9-ac456e…","""2023-11-27 07:14:18"""
20,"""Dmitriy Dryga""","""027a995b-0b07-42b9-975b-660b3b…","""2023-10-11 07:05:47"""
277,"""МВД Курганской области обновле…","""02c58e6c-1a8e-440e-8063-25e69a…","""2025-03-11 14:54:58"""
55,"""test test""","""03011373-3e11-4640-a47b-5adca3…","""2024-02-06 08:33:44"""
222,"""Денис""","""033c6419-baca-4475-9ce2-08e2ed…","""2024-10-23 18:10:29"""
63,"""Честная УК""","""045328cf-a192-40be-8ab4-fec1e5…","""2024-02-28 16:12:41"""


### Drop ch

In [ ]:
query_text = """
    DROP TABLE db1.uk_dir_partner_ch
    """
ch.query_run(query_text)

### Drop mv

In [ ]:
query_text = """
    DROP TABLE db1.uk_dir_partner_mv
    """
ch.query_run(query_text)